In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:

df = pd.read_csv('train.csv')

In [13]:

df.shape

(404290, 6)

In [14]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [15]:

new_df = df.sample(30000)

In [16]:

new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [17]:

new_df.duplicated().sum()

0

In [18]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
246434,"Do you think that you would like yourself, if ...","If you met yourself, would you like you?"
392072,What is the best way to earn money online?,Is Freelancer the best way to earn money online?
62435,What will be the repercussions of banning Rs 5...,What is the use of banning 500 and 1000 rupee ...
356084,What may be the possible future of the softwar...,How do software companies prevent piracy?
77552,How do you get a verified profile or page on F...,What process does Facebook use to verify pages?


In [21]:
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [22]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(30000, 6000)

In [23]:
temp_df

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
246434,0,0,0,0,0,0,0,0,0,0,...,0,3,0,0,0,1,0,0,0,0
392072,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
62435,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
356084,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
77552,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59077,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
350363,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33493,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
331565,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [25]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
246434,0,0,0,0,0,0,0,0,0,0,...,3,0,0,0,1,0,0,0,0,1
392072,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
62435,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
356084,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
77552,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values,test_size=0.2,random_state=1)

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7456666666666667

In [30]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test,y_pred)

0.7336666666666667